<a href="https://colab.research.google.com/github/pattichis/AdvancedPython/blob/main/FastPython2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast Python Execution using NumPy, Numba, GPUs, and AI prompting
The example demonstrates the use of for-loops, versus the use of NumPy, Numba on CPUs, and GPUs.

For the GPU example to run, click on Runtime and select a GPU.


Here is a typical speedup:
```
Problem size: N=1000, D=50
--------------------------------------------------
Pure Python: 4.682 s
NumPy:       0.028 s (Speedup: 164.6x)
Numba:       0.025 s (Speedup: 188.8x)
Numba CUDA:  0.006 s (Speedup: 803.0x)
All speedup tests passed! (min_speedup = 100.0x)
```

Note: NumPy calculates A LOT more than Numba.

The basic idea is to use a numba decorator:
```
import numba as nb

@nb.njit(parallel=True, fastmath=True)
def pairwise_dist_numba(X):
```

[Important paper to read (see section 2)](https://ieeexplore.ieee.org/document/11186485)<br>

[Cuda link](https://docs.nvidia.com/cuda/cuda-programming-guide/02-basics/writing-cuda-kernels.html#writing-cuda-kernels-sm-resource-example)


## Pseudocode for Pairwise Distance Calculation

```
function pairwise_distance(data_matrix):
    num_rows = number of rows in data_matrix
    distance_matrix = create an empty matrix of size (num_rows x num_rows) initialized with zeros

    for each row i from 0 to num_rows - 1:
        for each row j from 0 to num_rows - 1:
            squared_difference_sum = 0
            for each element k in the rows:
                difference = data_matrix[i][k] - data_matrix[j][k]
                squared_difference_sum = squared_difference_sum + (difference * difference)
            distance_matrix[i][j] = squared_difference_sum

    return distance_matrix
```

# Example code

In [10]:
import time
import numpy as np
import numba as nb
from numba import cuda

# ----------------------------
# Implementations
# ----------------------------

def pairwise_dist_python(X):
    n = len(X)
    d = len(X[0])
    D = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            s = 0.0
            for k in range(d):
                diff = X[i][k] - X[j][k]
                s += diff * diff
            D[i][j] = s
    return D


def pairwise_dist_numpy(X):
    G = X @ X.T
    sq_norms = np.diag(G)
    return sq_norms[:, None] - 2 * G + sq_norms[None, :]


@nb.njit(parallel=True, fastmath=True)
def pairwise_dist_numba(X):
    n, d = X.shape
    D = np.zeros((n, n))
    for i in nb.prange(n):
        for j in range(n):
            s = 0.0
            for k in range(d):
                diff = X[i, k] - X[j, k]
                s += diff * diff
            D[i, j] = s
    return D

@cuda.jit
def pairwise_dist_numba_cuda_kernel(X, D):
    # Global row and column indices for the current thread in the D matrix
    #  This is the setup for a 2-D indexing of threads on Cuda.
    #  All threads will run the same Cuda kernel listed here.
    #  Computes one 2-D index of D at a time.
    # printing breaks the kernel.
    row_idx, col_idx = cuda.grid(2)

    # Check bounds to avoid out-of-bounds memory access if grid size > actual matrix size
    if row_idx >= D.shape[0] or col_idx >= D.shape[1]:
        return

    # Thread indices within the block
    #   These are local variables of the running thread.
    #   Assuming that cuda.threadIdx comes from the index of running thread.
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # Block dimensions
    #    These must be initialized before calling this function. How?
    blockDim_x = cuda.blockDim.x
    blockDim_y = cuda.blockDim.y

    # Block's starting global row and column indices
    #  This defines the start of a single block that will be processed by a single core.
    block_start_row = cuda.blockIdx.x * blockDim_x
    block_start_col = cuda.blockIdx.y * blockDim_y

    # Feature dimension (D in NxD)
    D_features = X.shape[1]

    # Define shared memory arrays
    # s_X_row_tile will store X[block_start_row + tx, :] for all tx in the block
    # s_X_col_tile will store X[block_start_col + ty, :] for all ty in the block
    # The shape is (block_dimension, D_features)
    s_X_row_tile = cuda.shared.array(shape=(16, 50), dtype=X.dtype) # Using 16, 50 explicitly for clarity
    s_X_col_tile = cuda.shared.array(shape=(16, 50), dtype=X.dtype)

    # Cooperative loading into shared memory
    # Each thread in the block will help load data into shared memory.
    # For s_X_row_tile: thread (tx, 0) loads X[global_row, :] into s_X_row_tile[tx, :]
    if ty == 0 and block_start_row + tx < X.shape[0]:
        for k_feat in range(D_features):
            s_X_row_tile[tx, k_feat] = X[block_start_row + tx, k_feat]

    # For s_X_col_tile: thread (0, ty) loads X[global_col, :] into s_X_col_tile[ty, :]
    if tx == 0 and block_start_col + ty < X.shape[0]:
        for k_feat in range(D_features):
            s_X_col_tile[ty, k_feat] = X[block_start_col + ty, k_feat]

    # Synchronize threads to ensure all shared memory loads are complete before computation
    # 2. Straight-forward but makes the point about memory access into shared memory.
    # Marios: ALL threads complete their memory copies.
    cuda.syncthreads()

    # Compute the squared Euclidean distance using shared memory
    # 1. START your analysis here to establish notation. Then go backwards.
    s = 0.0
    if row_idx < X.shape[0] and col_idx < X.shape[0]: # Final check for valid indices
        for k_feat in range(D_features):
            # Access shared memory for computation
            diff = s_X_row_tile[tx, k_feat] - s_X_col_tile[ty, k_feat]
            s += diff * diff

        D[row_idx, col_idx] = s


def pairwise_dist_numba_cuda(X):
    # Copy data to device
    # Kevin: Copies to global memory in Cuda.
    # GPU: Global + shared + local (local done internally?)
    # Verify what .to_device() and device_array() mean
    X_device = cuda.to_device(X)
    D_device = cuda.device_array((X.shape[0], X.shape[0]), dtype=X.dtype)
    print(" ")
    print(f"GPU: X_device shape =  {X_device.shape} and D_device shape = {D_device.shape}")

    # Configure the blocks and grids
    # Kevin: Every block will have 16*16 threads.
    threadsperblock = (16, 16) #
    blockspergrid_x = (X.shape[0] + threadsperblock[0] - 1) // threadsperblock[0]
    blockspergrid_y = (X.shape[0] + threadsperblock[1] - 1) // threadsperblock[1]
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # See everything:
    print(f"threads per block = {threadsperblock}")
    print(f"blocks per grid x = {blockspergrid_x}")
    print(f"blocks per grid y = {blockspergrid_y}")
    print(f"blocks per grid   = {blockspergrid}")

    # Launch the kernel
    # FINAL. We need to cover this function call here. It looks strange and 2-D.
    # How do we derive all of the variables everything from here?
    pairwise_dist_numba_cuda_kernel[blockspergrid, threadsperblock](X_device, D_device)

    # Copy result back to host
    return D_device.copy_to_host()

# ----------------------------
# Timing helper
# ----------------------------

def timeit(fn, *args, repeat=3, warmup=False):
    if warmup:
        fn(*args)
    times = []
    for _ in range(repeat):
        t0 = time.perf_counter()
        fn(*args)
        times.append(time.perf_counter() - t0)
    return min(times)

# ----------------------------
# Benchmark
# ----------------------------

n, d = 1000, 50
np.random.seed(0)
X_np = np.random.randn(n, d).astype(np.float64)
X_py = X_np.tolist()

print(f"Problem size: N={n}, D={d}")
print("--------------------------------------------------")

# Warm up Numba (compilation happens here) - Redundant if warmup=True in timeit, but harmless.
pairwise_dist_numba(X_np)

# Warm up Numba CUDA (compilation happens here) - Redundant if warmup=True in timeit, but harmless.
# It's important to pass a copy to avoid side effects if the function modifies the input
if cuda.is_available():
    pairwise_dist_numba_cuda(X_np)

t_python = timeit(pairwise_dist_python, X_py, repeat=1, warmup=True)
t_numpy = timeit(pairwise_dist_numpy, X_np, warmup=True)
t_numba = timeit(pairwise_dist_numba, X_np, warmup=True)

print(f"Pure Python: {t_python:.3f} s")
print(f"NumPy:       {t_numpy:.3f} s (Speedup: {t_python/t_numpy:.1f}x)")
print(f"Numba:       {t_numba:.3f} s (Speedup: {t_python/t_numba:.1f}x)")

min_speedup = 100.0 # Define minimal speedup threshold

assert t_python / t_numpy >= min_speedup, f"NumPy speedup failed: {t_python / t_numpy:.1f}x < {min_speedup}x"
assert t_python / t_numba >= min_speedup, f"Numba speedup failed: {t_python / t_numba:.1f}x < {min_speedup}x"

if cuda.is_available():
    t_numba_cuda = timeit(pairwise_dist_numba_cuda, X_np, warmup=True)
    print(f"Numba CUDA:  {t_numba_cuda:.3f} s (Speedup: {t_python/t_numba_cuda:.1f}x)")
    assert t_python / t_numba_cuda >= min_speedup, f"Numba CUDA speedup failed: {t_python / t_numba_cuda:.1f}x < {min_speedup}x"
    print(f"All speedup tests passed! (min_speedup = {min_speedup}x)")
else:
    print("Numba CUDA:  GPU not available, skipping CUDA benchmark and speedup test.")
    print(f"NumPy and Numba speedup tests passed! (min_speedup = {min_speedup}x)")

Problem size: N=1000, D=50
--------------------------------------------------
 
GPU: X_device shape =  (1000, 50) and D_device shape = (1000, 1000)
threads per block = (16, 16)
blocks per grid x = 63
blocks per grid y = 63
blocks per grid   = (63, 63)
Pure Python: 3.577 s
NumPy:       0.012 s (Speedup: 304.5x)
Numba:       0.006 s (Speedup: 553.0x)
 
GPU: X_device shape =  (1000, 50) and D_device shape = (1000, 1000)
threads per block = (16, 16)
blocks per grid x = 63
blocks per grid y = 63
blocks per grid   = (63, 63)
 
GPU: X_device shape =  (1000, 50) and D_device shape = (1000, 1000)
threads per block = (16, 16)
blocks per grid x = 63
blocks per grid y = 63
blocks per grid   = (63, 63)
 
GPU: X_device shape =  (1000, 50) and D_device shape = (1000, 1000)
threads per block = (16, 16)
blocks per grid x = 63
blocks per grid y = 63
blocks per grid   = (63, 63)
 
GPU: X_device shape =  (1000, 50) and D_device shape = (1000, 1000)
threads per block = (16, 16)
blocks per grid x = 63
block